FINAL CODE FOR SEGMENTATION:

In [ ]:
pip install yt_dlp

In [ ]:
import yt_dlp

def download_audio(video_url):
    ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '/content/audio.mp3',
    'cookiefile': '/content/cookies (3).txt',  # Update this path
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
  }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    return "/content/audio.mp3"

video_url = input("Enter YouTube video URL: ")
file_path = download_audio(video_url)
print(f"Downloaded audio: {file_path}")

In [ ]:
import os

# Check the files in /content directory
print("Files in content:", os.listdir("/content"))

# Fix the file name if needed
old_path = "/content/audio.mp3.mp3"
new_path = "/content/audio.mp3"

if os.path.exists(old_path):
    os.rename(old_path, new_path)
    print(f"Renamed: {old_path} → {new_path}")
else:
    print("Audio file not found!")

In [ ]:
import IPython.display as ipd

# Set the correct file path
audio_path = "/content/audio.mp3"

# Play the audio with the correct sample rate (e.g., 44100 Hz)
ipd.Audio(audio_path, rate=44100)

In [ ]:
import os
import whisper
from pydub import AudioSegment

def convert_audio_to_wav(audio_file):
    """Converts an MP3 audio file to WAV format for better transcription."""
    wav_file = "audio.wav"
    if audio_file.endswith(".mp3"):
        try:
            AudioSegment.from_mp3(audio_file).export(wav_file, format="wav")
            return wav_file
        except Exception as e:
            return f"Error converting to WAV: {str(e)}"
    return audio_file  # If already WAV, return as is

def transcribe_audio_with_whisper(audio_path):
    """Transcribes audio using OpenAI's Whisper model."""
    model = whisper.load_model("base")  # Options: "tiny", "base", "small", "medium", "large"

    print("🔹 Transcribing with Whisper... (this may take some time)")

    result = model.transcribe(audio_path)
    return result["text"]

def save_transcript_to_file(transcript, filename="transcript.txt"):
    """Saves the transcript to a text file."""
    with open(filename, "w", encoding="utf-8") as file:
        file.write(transcript)
    print(f"✅ Transcript saved to {filename}")

# Example usage
if __name__ == "__main__":
    audio_file = input("Enter the path to your audio file (MP3/WAV): ")

    if not os.path.exists(audio_file):
        print("❌ Error: File not found.")
    else:
        wav_file = convert_audio_to_wav(audio_file)

        if "Error" in wav_file:
            print(wav_file)
        else:
            transcript = transcribe_audio_with_whisper(wav_file)
            print("\n🔹 Transcript:\n", transcript)

            # Save transcript
            save_transcript_to_file(transcript, "transcript.txt")

            # Cleanup WAV file if it was converted
            if wav_file != audio_file:
                os.remove(wav_file)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopword')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

def read_transcript(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            transcript = file.read()
        return transcript
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

def split_into_sentences(transcript):
    return sent_tokenize(transcript)

def calculate_cosine_similarity(sentences):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    cosine_sim = cosine_similarity(tfidf_matrix)
    return cosine_sim

def segment_sentences(sentences, cosine_sim, threshold=0.5, min_sentences=5):
    visited = [False] * len(sentences)
    segments = []

    for i in range(len(sentences)):
        if not visited[i]:
            segment = [sentences[i]]
            visited[i] = True

            for j in range(i + 1, len(sentences)):
                if not visited[j] and cosine_sim[i][j] >= threshold:
                    segment.append(sentences[j])
                    visited[j] = True

            segments.append(segment)

    # Merge smaller segments
    merged_segments = []
    temp_segment = []

    for segment in segments:
        temp_segment.extend(segment)
        if len(temp_segment) >= min_sentences:
            merged_segments.append(temp_segment)
            temp_segment = []

    if temp_segment:
        if merged_segments:
            merged_segments[-1].extend(temp_segment)
        else:
            merged_segments.append(temp_segment)

    return merged_segments

def print_segments(segments):
    for idx, segment in enumerate(segments, start=1):
        print(f"\nSegment {idx}:")
        for sentence in segment:
            print(f" - {sentence}")

def save_segments_to_file(segments, output_file):
    try:
        with open(output_file, 'w', encoding='utf-8') as file:
            for idx, segment in enumerate(segments, start=1):
                file.write(f"Segment {idx}:\n")
                for sentence in segment:
                    file.write(f" - {sentence}\n")
                file.write("\n")
        print(f"Segmented output saved to {output_file}")
    except Exception as e:
        print(f"Error saving file: {e}")

def process_transcript(file_path, threshold=0.5, min_sentences=5, output_file='segmented_output.txt'):
    transcript = read_transcript(file_path)
    if transcript is None:
        return

    sentences = split_into_sentences(transcript)
    print("Sentences extracted:")
    for i, sentence in enumerate(sentences):
        print(f"S{i + 1}: {sentence}")

    cosine_sim = calculate_cosine_similarity(sentences)

    segments = segment_sentences(sentences, cosine_sim, threshold, min_sentences)

    print("\nSegmented Sentences:")
    print_segments(segments)

    save_segments_to_file(segments, output_file)

file_path = '/content/transcript.txt'  # Replace with your file path
output_file = '/content/segmented_output.txt'
process_transcript(file_path, threshold=0.15, min_sentences=5, output_file=output_file)

Keyword Generation

In [ ]:
!pip install gensim spacy nltk

Using LDA

In [ ]:
import re
import gensim
import spacy
import nltk
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords

# Download stopwords if not available
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    doc = nlp(text.lower())  # Convert to lowercase and tokenize
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]
    return tokens

# Read the transcript from file
input_file = "/content/segmented_output.txt"  # Change this to your actual file name
with open(input_file, "r", encoding="utf-8") as file:
    transcript = file.read()

# Preprocess each segment in the transcript
processed_texts = [preprocess_text(segment) for segment in transcript.split("\n") if segment.strip()]

# Create Dictionary and Corpus for LDA
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Train LDA Model
num_topics = 3  # Adjust based on requirement
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Display Extracted Topics
print("\nExtracted Topics:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx + 1}: {topic}")

In [ ]:
import re
import gensim
import spacy
import nltk
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords

# Download stopwords if not available
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    doc = nlp(text.lower())  # Convert to lowercase and tokenize
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]
    return tokens

# Read the transcript from file
input_file = "/content/segmented_output.txt"  # Change this to your actual file name
with open(input_file, "r", encoding="utf-8") as file:
    transcript = file.readlines()

# Preprocess each segment
processed_texts = [preprocess_text(segment) for segment in transcript if segment.strip()]

# Create Dictionary and Corpus for LDA
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Train LDA Model
num_topics = 5  # Adjust based on requirement
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Extract the dominant topic for each segment and save to file
output_file = "output_topics.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for i, bow in enumerate(corpus):
        topic_distribution = lda_model.get_document_topics(bow)
        dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]  # Get the most probable topic
        topic_keywords = lda_model.print_topic(dominant_topic)

        out_file.write(f"Segment {i + 1}:\n")
        out_file.write(f"Topic: {topic_keywords}\n\n")

print(f"Extracted topics saved to {output_file}")


In [ ]:
import re
import gensim
import spacy
import nltk
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords

# Download stopwords if not available
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess text
def preprocess_text(text):
    """Cleans and tokenizes text for topic modeling"""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    doc = nlp(text.lower())  # Convert to lowercase and tokenize
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]
    return tokens

# Read the transcript file
input_file = "/content/segmented_output (2).txt"  # Change this to your actual file name
with open(input_file, "r", encoding="utf-8") as file:
    transcript = file.read()

# Split transcript into segments using a pattern like 'Segment X:' or other logical markers
segments = re.split(r'Segment \d+:', transcript)[1:]  # Removes empty first split

# Preprocess each full segment
processed_texts = [preprocess_text(segment.strip()) for segment in segments if segment.strip()]

# Create Dictionary and Corpus for LDA
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Train LDA Model
num_topics = min(len(processed_texts), 5)  # Set topic count based on available data
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Extract the top keyword for each full segment
output_file = "top_keywords.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for i, bow in enumerate(corpus):
        topic_distribution = lda_model.get_document_topics(bow)
        dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]  # Get the most probable topic
        topic_keywords = lda_model.print_topic(dominant_topic)

        # Extract the highest probability keyword
        keywords = topic_keywords.split(" + ")
        top_keyword = max(keywords, key=lambda x: float(x.split("*")[0]))  # Get highest weight word
        top_word = top_keyword.split("*")[1].replace("'", "").strip()  # Extract the actual word

        out_file.write(f"Segment {i + 1}: {top_word}\n")

print(f"Top keywords saved to {output_file}")

In [ ]:
import re
import gensim
import spacy
import nltk
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords

# Download stopwords if not available
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess text
def preprocess_text(text):
    """Cleans and tokenizes text for topic modeling"""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    doc = nlp(text.lower())  # Convert to lowercase and tokenize
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]
    return tokens

# Read the transcript file
input_file = "/content/segmented_output (2).txt"  # Change this to your actual file name
with open(input_file, "r", encoding="utf-8") as file:
    transcript = file.read()

# Split transcript into segments using 'Segment X:' markers
segments = re.split(r'Segment \d+:', transcript)[1:]  # Removes empty first split

# Preprocess each full segment
processed_texts = [preprocess_text(segment.strip()) for segment in segments if segment.strip()]

# Create Dictionary and Corpus for LDA
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Train LDA Model
num_topics = min(len(processed_texts), 5)  # Set topic count based on available data
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Extract the top 3 keywords for each full segment
output_file = "top_keywords2.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for i, bow in enumerate(corpus):
        topic_distribution = lda_model.get_document_topics(bow)
        dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]  # Get the most probable topic
        topic_keywords = lda_model.print_topic(dominant_topic)

        # Extract top 3 highest probability keywords
        keywords = topic_keywords.split(" + ")
        top_keywords = sorted(keywords, key=lambda x: float(x.split("*")[0]), reverse=True)[:3]
        top_words = [word.split("*")[1].replace("'", "").strip() for word in top_keywords]

        out_file.write(f"Segment {i + 1}: {', '.join(top_words)}\n")

print(f"Top keywords saved to {output_file}")

Using KeyBERT

In [ ]:
pip install keybert

In [ ]:
import re
import spacy
import nltk
from keybert import KeyBERT
from nltk.corpus import stopwords

# Download stopwords if not available
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Initialize KeyBERT model
kw_model = KeyBERT()

# Function to preprocess text
def preprocess_text(text):
    """Cleans text and removes unnecessary characters."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    doc = nlp(text.lower())  # Convert to lowercase and tokenize
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]
    return " ".join(tokens)  # Return as cleaned text

# Read the transcript file
input_file = "/content/segmented_output (2).txt"  # Change this to your actual file name
with open(input_file, "r", encoding="utf-8") as file:
    transcript = file.read()

# Split transcript into segments using 'Segment X:' markers
segments = re.split(r'Segment \d+:', transcript)[1:]  # Removes empty first split

# Preprocess each segment
cleaned_segments = [preprocess_text(segment.strip()) for segment in segments if segment.strip()]

# Extract top 3 keywords using KeyBERT
output_file = "top_keywords.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for i, segment in enumerate(cleaned_segments):
        keywords = kw_model.extract_keywords(segment, keyphrase_ngram_range=(1, 1), top_n=3)

        # Get top 3 keywords
        top_words = [word[0] for word in keywords]

        out_file.write(f"Segment {i + 1}: {', '.join(top_words)}\n")

print(f"Top keywords saved to {output_file}")

Linkedin Transcript

In [ ]:
import yt_dlp

def download_audio(video_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'linkedin_audio.mp3',  # Save file with a meaningful name
        'cookiefile': '/content/cookies (4).txt',  # Ensure you have LinkedIn authentication
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    return "linkedin_audio.mp3"

video_url = input("Enter LinkedIn video URL: ")
file_path = download_audio(video_url)
print(f"Downloaded audio: {file_path}")


In [ ]:
!pip install -U yt-dlp


In [ ]:
!yt-dlp --cookies-from-browser chrome "https://www.linkedin.com/posts/guillermoflor_elon-musk-wants-the-workplace-to-be-vibrant-activity-7208090955906707457-YQ__?utm_source=share&utm_medium=member_desktop&rcm=ACoAAC472oABr5i8229nuG8UuSMZV1GiCnsrmuE"

In [ ]:
yt-dlp --cookies-from-browser chrome "https://www.linkedin.com/posts/guillermoflor_elon-musk-wants-the-workplace-to-be-vibrant-activity-7208090955906707457-YQ__?utm_source=share&utm_medium=member_desktop&rcm=ACoAAC472oABr5i8229nuG8UuSMZV1GiCnsrmuE"

TRANSCRIPT SEGMENTATION

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Ensure necessary resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalnum() and word not in stop_words]

def calculate_similarity(segment_words, new_sentence_words):
    vectorizer = TfidfVectorizer()
    combined_text = [" ".join(segment_words), " ".join(new_sentence_words)]
    tfidf_matrix = vectorizer.fit_transform(combined_text)
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

def segment_transcript(file_path, output_path, threshold=0.5):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    lines = [line.strip() for line in lines if line.strip()]
    segments = []
    i = 0

    while i < len(lines):
        segment = lines[i:i+5]
        segment_words = preprocess_text(" ".join(segment))

        new_segment = segment[:]
        i += 5

        while i < len(lines):
            new_sentence = lines[i]
            new_sentence_words = preprocess_text(new_sentence)

            similarity = calculate_similarity(segment_words, new_sentence_words)

            if similarity >= threshold:
                new_segment.append(new_sentence)
                segment_words.extend(new_sentence_words)
                i += 1
            else:
                break

        segments.append(" ".join(new_segment))

        # Start the next segment from the current sentence (not skipping lines)

    with open(output_path, 'w', encoding='utf-8') as output_file:
        for idx, segment in enumerate(segments):
            output_file.write(f"Segment {idx + 1}:\n{segment}\n\n")

    return segments

# Example usage
file_path = "/content/transcript (15).txt"  # Path to the input .txt file
output_path = "segmented_transcript.txt"  # Path to the output .txt file
segments = segment_transcript(file_path, output_path)
for idx, segment in enumerate(segments):
    print(f"Segment {idx + 1} saved to file.")

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Ensure necessary resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalnum() and word not in stop_words]

def calculate_similarity(reference_text, new_sentence):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([" ".join(reference_text), new_sentence])
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

def segment_transcript(file_path, output_path, threshold=1.0):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]

    segments = []
    i = 0

    while i < len(lines):
        segment = lines[i:i+5]  # First 5 lines as an initial segment
        segment_words = preprocess_text(" ".join(segment))

        new_segment = segment[:]
        i += 5  # Move forward

        while i < len(lines):
            new_sentence = lines[i]
            new_sentence_words = preprocess_text(new_sentence)

            similarity = calculate_similarity(segment_words, " ".join(new_sentence_words))

            if similarity >= threshold:
                new_segment.append(new_sentence)
                i += 1
            else:
                break  # Start a new segment

        segments.append(" ".join(new_segment))

    with open(output_path, 'w', encoding='utf-8') as output_file:
        for idx, segment in enumerate(segments):
            output_file.write(f"Segment {idx + 1}:\n{segment}\n\n")

    return segments

# Example usage
file_path = "/content/transcript (15).txt"
output_path = "segmented_transcript.txt"
segments = segment_transcript(file_path, output_path, threshold=0.6)

for idx, segment in enumerate(segments):
    print(f"Segment {idx + 1} saved to file.")

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Function to clean text by removing stopwords
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    return ' '.join([word for word in words if word.isalnum() and word not in stop_words])

# Read transcript
def read_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

# Function to group sentences into segments and save them
def segment_transcript(sentences, output_file):
    segments = []
    current_segment = sentences[:5]  # First 5 lines as the initial segment
    cleaned_segment_words = clean_text(' '.join(current_segment))  # Remove stopwords

    for i in range(5, len(sentences)):
        sentence = sentences[i]
        cleaned_sentence = clean_text(sentence)

        if not cleaned_sentence:  # Skip empty sentences after stopword removal
            continue

        # Calculate similarity
        vectorizer = TfidfVectorizer().fit_transform([cleaned_segment_words, cleaned_sentence])
        similarity = cosine_similarity(vectorizer)[0, 1]

        if similarity > 0.5:  # Threshold to decide if it belongs in the same segment
            current_segment.append(sentence)
        else:
            segments.append(current_segment)  # Save the current segment
            current_segment = [sentence] + sentences[i+1:i+5]  # New segment with next 4 lines
            cleaned_segment_words = clean_text(' '.join(current_segment))  # Recalculate cleaned words

    segments.append(current_segment)  # Append the last segment

    # Save segmented transcript to a new file
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx, segment in enumerate(segments):
            file.write(f"Segment {idx + 1}:\n")
            file.write("".join(segment) + "\n")
            file.write("-" * 50 + "\n")

    return segments

# Example usage
if __name__ == "__main__":
    transcript_lines = read_transcript("/content/transcript (15).txt")
    segmented_text = segment_transcript(transcript_lines, "segmented_transcript.txt")

    for idx, segment in enumerate(segmented_text):
        print(f"Segment {idx + 1}:")
        print("".join(segment))
        print("-" * 50)

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Function to clean text by removing stopwords
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    return ' '.join([word for word in words if word.isalnum() and word not in stop_words])

# Read transcript
def read_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

# Function to group sentences into segments and save them
def segment_transcript(sentences, output_file):
    segments = []
    i = 0  # Pointer to track position in sentences

    while i < len(sentences):
        current_segment = sentences[i:i+5]  # First 5 lines as the initial segment
        cleaned_segment_words = clean_text(' '.join(current_segment))  # Remove stopwords
        i += 5  # Move the pointer forward

        while i < len(sentences):
            cleaned_sentence = clean_text(sentences[i])

            if not cleaned_sentence:  # Skip empty sentences after stopword removal
                i += 1
                continue

            # Calculate similarity
            vectorizer = TfidfVectorizer().fit_transform([cleaned_segment_words, cleaned_sentence])
            similarity = cosine_similarity(vectorizer)[0, 1]

            if similarity > 0.5:  # Threshold to decide if it belongs in the same segment
                current_segment.append(sentences[i])
                i += 1  # Move to the next sentence
            else:
                break  # Stop adding to this segment, start a new one

        segments.append(current_segment)

    # Save segmented transcript to a new file
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx, segment in enumerate(segments):
            file.write(f"Segment {idx + 1}:\n")
            file.write("".join(segment) + "\n")
            file.write("-" * 50 + "\n")

    return segments

# Example usage
if __name__ == "__main__":
    transcript_lines = read_transcript("/content/transcript (15).txt")
    segmented_text = segment_transcript(transcript_lines, "segmented_transcript.txt")

    for idx, segment in enumerate(segmented_text):
        print(f"Segment {idx + 1}:")
        print("".join(segment))
        print("-" * 50)

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Function to clean text by removing stopwords
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    return ' '.join([word for word in words if word.isalnum() and word not in stop_words])

# Read transcript
def read_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

# Function to group sentences into segments and save them
def segment_transcript(sentences, output_file):
    segments = []
    i = 0  # Pointer to track position in sentences

    while i < len(sentences):
        current_segment = sentences[i:i+5]  # First 5 lines as the initial segment
        cleaned_segment_words = clean_text(' '.join(current_segment))  # Remove stopwords
        i += 5  # Move the pointer forward

        while i < len(sentences):
            cleaned_sentence = clean_text(sentences[i])

            if not cleaned_sentence:  # Skip empty sentences after stopword removal
                i += 1
                continue

            # Calculate similarity
            vectorizer = TfidfVectorizer().fit_transform([cleaned_segment_words, cleaned_sentence])
            similarity = cosine_similarity(vectorizer)[0, 1]

            if similarity > 0.2:  # Threshold to decide if it belongs in the same segment
                current_segment.append(sentences[i])
                i += 1  # Move to the next sentence
            else:
                break  # Stop adding to this segment, start a new one

        segments.append(current_segment)
        i += 5  # Move the pointer forward to ensure the next segment starts correctly

    # Save segmented transcript to a new file
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx, segment in enumerate(segments):
            file.write(f"Segment {idx + 1}:\n")
            file.write("".join(segment) + "\n")
            file.write("-" * 50 + "\n")

    return segments

# Example usage
if __name__ == "__main__":
    transcript_lines = read_transcript("/content/transcript (15).txt")
    segmented_text = segment_transcript(transcript_lines, "segmented_transcript.txt")

    for idx, segment in enumerate(segmented_text):
        print(f"Segment {idx + 1}:")
        print("".join(segment))
        print("-" * 50)


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Function to clean text by removing stopwords
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    return ' '.join([word for word in words if word.isalnum() and word not in stop_words])

# Read transcript file
def read_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return sent_tokenize(file.read())  # Use sentence tokenizer instead of readlines

# Function to segment transcript
def segment_transcript(sentences, output_file):
    segments = []
    i = 0  # Pointer to track position

    while i < len(sentences):
        current_segment = [sentences[i]]  # Start with first sentence
        cleaned_segment = clean_text(sentences[i])
        i += 1  # Move forward

        while i < len(sentences):
            cleaned_sentence = clean_text(sentences[i])

            if not cleaned_sentence:  # Skip empty cleaned sentences
                i += 1
                continue

            # Vectorize segment and new sentence
            vectorizer = TfidfVectorizer()
            vectors = vectorizer.fit_transform([cleaned_segment, cleaned_sentence])
            similarity = cosine_similarity(vectors)[0, 1]

            if similarity > 0.9:  # Threshold for grouping
                current_segment.append(sentences[i])
                cleaned_segment += " " + cleaned_sentence  # Update segment content
                i += 1  # Move to next sentence
            else:
                break  # Stop adding, create a new segment

        segments.append(current_segment)

    # Save segmented transcript
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx, segment in enumerate(segments):
            file.write(f"Segment {idx + 1}:\n")
            file.write(" ".join(segment) + "\n")
            file.write("-" * 50 + "\n")

    return segments

# Example usage
if __name__ == "__main__":
    transcript_sentences = read_transcript("/content/transcript (15).txt")
    segmented_text = segment_transcript(transcript_sentences, "segmented_transcript.txt")

    for idx, segment in enumerate(segmented_text):
        print(f"Segment {idx + 1}:")
        print(" ".join(segment))
        print("-" * 50)


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download required data
nltk.download('punkt')

# Read transcript file
def read_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return sent_tokenize(file.read())  # Tokenize entire text into sentences

# Function to segment transcript using similarity
def segment_transcript(sentences, output_file, threshold=0.9):
    segments = []
    i = 0  # Pointer to track position

    # Fit a single TF-IDF vectorizer on all sentences for consistency
    vectorizer = TfidfVectorizer(stop_words="english")
    sentence_vectors = vectorizer.fit_transform(sentences)  # Vectorize entire text

    while i < len(sentences):
        current_segment = [sentences[i]]  # Start with the first sentence
        segment_indices = [i]
        i += 1  # Move pointer forward

        while i < len(sentences):
            # Compute cosine similarity between last sentence of segment and next sentence
            similarity = cosine_similarity(sentence_vectors[segment_indices[-1]], sentence_vectors[i])[0, 0]

            if similarity >= threshold:  # If similar, add to the segment
                current_segment.append(sentences[i])
                segment_indices.append(i)  # Track index of included sentence
                i += 1
            else:
                break  # Stop and start a new segment

        segments.append(current_segment)  # Store the formed segment

    # Save segmented transcript
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx, segment in enumerate(segments):
            file.write(f"Segment {idx + 1}:\n")
            file.write(" ".join(segment) + "\n")
            file.write("-" * 50 + "\n")

    return segments

# Example usage
if __name__ == "__main__":
    transcript_sentences = read_transcript("/content/transcript (15).txt")
    segmented_text = segment_transcript(transcript_sentences, "segmented_transcript.txt")

    for idx, segment in enumerate(segmented_text):
        print(f"Segment {idx + 1}:")
        print(" ".join(segment))
        print("-" * 50)

In [ ]:
# prompt: 1. Read the transcript
# 2. Separate each line
# 3. By default we have the first 5 lines in segment 1
# 4. Then we will remove the stop words from that segment and put the remaining words in a new list
# 5. Now we will start with the comparison with nest lines one by one
# 6. Remove the stop words from the sentences below them and compare with those of the list from segment 1
# 7. If the stop words from the sentence 6 are similar to those of segment 1 (Finding the cosine similarity) Put them in segment 1 else if they are not similar then they become the first line for the next segment.
# 8. The next segment will consists of this sentence 6 and the next 4 chronological sentences should also be kept in segment 2.
# 9. Now we will repeat the steps 4-8 for the rest of the transcript.

import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

# Download required data
nltk.download('punkt')
nltk.download('stopwords')

# Function to clean text by removing stopwords
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text.lower())
    return ' '.join([word for word in words if word.isalnum() and word not in stop_words])

# Read transcript file
def read_transcript(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return sent_tokenize(file.read())

# Function to segment transcript using similarity
def segment_transcript(sentences, output_file, threshold=1):
    segments = []
    i = 0

    # Fit a single TF-IDF vectorizer on all sentences for consistency
    vectorizer = TfidfVectorizer(stop_words="english")
    sentence_vectors = vectorizer.fit_transform(sentences)

    while i < len(sentences):
        current_segment = [sentences[i]]
        segment_indices = [i]
        i += 1

        while i < len(sentences):
            similarity = cosine_similarity(sentence_vectors[segment_indices[-1]], sentence_vectors[i])[0, 0]

            if similarity >= threshold:
                current_segment.append(sentences[i])
                segment_indices.append(i)
                i += 1
            else:
                break

        segments.append(current_segment)

    # Save segmented transcript
    with open(output_file, 'w', encoding='utf-8') as file:
        for idx, segment in enumerate(segments):
            file.write(f"Segment {idx + 1}:\n")
            file.write(" ".join(segment) + "\n")
            file.write("-" * 50 + "\n")

    return segments

# Example usage
if __name__ == "__main__":
    transcript_sentences = read_transcript("/content/transcript (15).txt")
    segmented_text = segment_transcript(transcript_sentences, "segmented_transcript.txt")

    for idx, segment in enumerate(segmented_text):
        print(f"Segment {idx + 1}:")
        print(" ".join(segment))
        print("-" * 50)


In [ ]:
#Using LLM

In [ ]:
pip uninstall bitsandbytes -y

In [ ]:
pip install bitsandbytes --no-cache-dir

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
import torch

In [ ]:
# Ensure GPU is available
assert torch.cuda.is_available(), "GPU is not available! Check your CUDA installation."

In [ ]:
pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [ ]:
with open("/content/transcript (15).txt", "r", encoding="utf-8") as file:
    transcript_text = file.read()

In [ ]:
inputs = [
    {
        "role": "system",
        "content": "You are a helpful assistant tasked with segmenting a long video lecture transcript into clear, distinct topic sections."
    },
    {
        "role": "user",
        "content": f"You have been provided with a long video lecture transcript. Your task is Read the transcript, split it into lines, group the first five lines into segment 1, remove stop words, store the remaining words in a list, compare each following line (after stop-word removal) using cosine similarity, add similar lines to segment 1, else start a new segment with the unmatched line and the next four lines, then repeat this process for the rest of the transcript."
    },
    {
        "role": "assistant",
        "content": "Please segment the transcript provided below into clear sections without altering the original content. Ensure that no new text is generated or sentence structure is modified."
    },
    {
        "role": "user",
        "content": transcript_text
    }
]

In [ ]:
a = pipe(inputs, max_length=500000)

In [ ]:
#The Method we discussed

In [ ]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    words = nltk.word_tokenize(text.lower())
    return ' '.join([word for word in words if word.isalnum() and word not in stop_words])

def segment_transcript(transcript):
    lines = transcript.strip().split('\n')
    segments = []
    current_segment = lines[:5]  # First 5 lines
    processed_segment = [preprocess_text(line) for line in current_segment]
    vectorizer = TfidfVectorizer()
    segment_vectors = vectorizer.fit_transform(processed_segment)

    i = 5
    while i < len(lines):
        sentence = lines[i]
        processed_sentence = preprocess_text(sentence)
        sentence_vector = vectorizer.transform([processed_sentence])

        similarity_scores = cosine_similarity(sentence_vector, segment_vectors).flatten()
        max_similarity = np.max(similarity_scores) if len(similarity_scores) > 0 else 0

        if max_similarity > 0.5:  # Threshold for similarity
            current_segment.append(sentence)
            processed_segment.append(processed_sentence)
            segment_vectors = vectorizer.fit_transform(processed_segment)
        else:
            segments.append(current_segment)  # Store completed segment
            current_segment = [sentence] + lines[i+1:i+5]  # Start new segment
            processed_segment = [preprocess_text(line) for line in current_segment]
            segment_vectors = vectorizer.fit_transform(processed_segment)
            i += 4  # Skip next 4 lines as they are in the new segment

        i += 1

    if current_segment:
        segments.append(current_segment)  # Append last segment

    return segments

# Example usage
transcript = """ So, sir, we know that India has seen a huge revolution with digital payments. We all thought that India is a place, at least the West thought that India is a place where many people do not get a square mill, right. That was an narrative some 30 years ago. And not many are literate, people cannot read. But then we have now shown that digital payments, number one is India, while people thought that it wouldn't even come to top 50, let alone number one. I think immediately after UPI the next big revolution, personally I think is in education. And the complete homework for this has happened in the form of NEP, the documentation of which many of us have read and realized that it is too good to be true. So, followed by which we got NCRF framework done, which is a national creative framework, surrounding which we will be discussing today. So, my question is, do you think NCRF plus NEP put together will be the next big revolution after UPI in India? Absolutely. And why I think so is because in education the last policy came up many many years ago that was in 1986, which was slightly tweaked in 1982. And thereafter so many changes have happened in the real world, so many changes have happened in the requirement of the industry, requirement of manpower for the industry. However there were no corresponding changes which happened in education system. So, therefore I feel that this was the right time when we brought in the education policy 2020, honorable prime minister declared it open in July 2020. And we recently celebrated the fourth anniversary of NEP 2020. NCRF has been brought to implement the intent of 2020, 2020 is a policy, NEP 2020 is a policy. And for implementing a policy you need a framework. Now, why we call it a framework? We call it a framework because this is very flexible. This allows you all the innovation, the way you educate your students. Still it provides you the basic guidelines, the framework, the outer layer it provides. And that layer is mostly the enabling layer. That is such a enabling layer that it has broken the shackles which were there in the education sector. So, yes it is a big revolution and this is going to change the way we have been educating our kids and this will be game changer for rest of the world as well. So, sir I think let us go with this example of let us say I started off living in a small 2BHK apartment and I slowly developed my kitchen to add to my taste buds in a way that it is convenient for me. And one fine day you came and you changed my kitchen completely. I was using a bicycle, I moved to a scooter and a car and now we are asking me to fly and giving me an aircraft. NEP sounds more like that for me. How do I do the transition? I fear that I will crash if I use a aeroplane without training. I am talking about all the teachers in the country, all the schools in the country. We have been driving buses at max. Now we should fly how do we do this? Okay. Look at the requirement of the industry. The requirement of the industry has been moving very fast. The technology is emerging every day and the industry is moving with that speed. So, when a student is coming out of your institute and is going out in the market, he finds that whatever he has been taught has no relevance to the real life world. When industry is moving that fast, when the requirement is moving that fast, don't you think it is important for us to change? Yes, how long can we wait? Yes, it will take a lot of effort for every one of us to adapt to this change. But this change is going to be not only beneficial but also very facilitated for all of us, very liberating for all of us. This is going to be highly liberative and choice based system. There are number of choices which are available to you, which are available to every student. Yes, when we introduce a new system, we have to really create new things, create new ways of doing things, learn something new. But once we learn it, there is no limit to innovation and creativity in the student. There is going to be a learning curve here, it is going to be difficult to fly a fighter jet, but it is not really difficult to fly. Because all of us, we have that kind of vision, that kind of innovative minds. The only thing is that we were not translating our vision and innovative minds into the education sector. Whereas we are applying it elsewhere, everywhere else, we are moving very fast. It is the only education and I think the education sector is the smartest sector, which is very good at learning. And I am very sure that learning this new way is going to be fun for everyone. And I can tell you that already a number of institutions have adopted the NEP and NCRF to varying degrees. And wherever they have adopted NCRF, they are much happier. And students are even happier. So, then I have spoken to many academics. Most of us do not understand NEP, let alone NCRF, which is the next step on how to implement NEP. If I can request you to give me an elevator pitch for NEP and then an elevator pitch for NCRF for the exclusive reason that I want to copy past it and then convince people, what is that elevator pitch for NEP? Okay. See, if I talk about one simple line on NEP and NCRF, that is it is creditization of all learning. It allows for creditization of all learning whether in academics, or in skilling or in experiential learning. And all these three kinds of skillings are required in the real world. So, when I am preparing the students for next generation for Wixit Bharat 2047, it is but natural that I would like to skill them as well. And people who are already skilled or already in the professional area, now there the experiential learning would play a big part. So, creditization of all learning, academics, skills and experiential learning, that is one. Second I would say is multi-displarity, inter-displarity and trans-displarity, that is that is how the world works. You show me a single piece of technology which has been created by single department. Good question. So, therefore, all of us we have to work in tandem with each other. So, that is the second holistic education. We do not have now the concept of co-curriculum and extracurricular. Everything is curricular. It is the way you define your curricular structure. You have place for everything, all kinds of learning are being characterized. Including learning of short skills, employability skills, life skills, your hand skills, your cognitive skills. Now, for everything you define it as learning. Right. And then, multiple and multiple exit anytime you can get into education, anytime you can go out. So, all very flexible. So, that there is no dropout. There is no dropout. So, these three things coupled with use of technology and multilimilism. So, this defines basically the NEPN and CRF. In short. Right. Perfect. So, one problem that we have been watching, India, suffer and struggle is this typical mindset of I want my daughter to be a doctor. I want my son to be an engineer. Do you think if we create a gamified approach to create systems, everybody will come to the same passage and would like to go to the same destination. How do we achieve multi-disciplinary, trans-disciplinary? Everybody wants to get into let's say CSE discipline or medicine discipline. Okay. You look at the kind of the requirement of infrastructure in India has today. No, whatever infrastructure we created in last 70, 75 years, we are going to double that infrastructure in next 10, 11 years. And when we approach 20, 47, maybe it is tripled. So, I need more number of civil engineers. I need more number of mechanical engineers, more number of electrical engineers, more number of other branches, even liberal arts, social sciences. If I want to be an evolved society, I would need a proper mix of all kinds of educated people. So, this is just a bad charm that people are just joining CSE. And there are no students or there are very less number of students in other branches. And we already seeing the fallout of this. You can see the fallout. You can see how many engineers are there for designing our bridges. You see what happened last six months. So, we cannot really ignore that. Though this is a temporary phenomena that wherever people feel that there are more number of jobs, everyone is running towards one particular branch. This branch is equally important because AI is also very important. But along with it, we have other branches. Just now along with mechanical engineering and civil engineering, we must also teach computer science that at the time. And that is where the interdisciplinary and transdisciplinary approach would come. You imagine a doctor. Today, if a doctor knows only about his own profession, he is unable to operate any of the new technology machines. Any laser based machines. Any automated operating machines, robotic processes, they are unable to operate. So, they will have to learn the skills in the other areas as well. And all these are skills. He need not learn about the theory of the robotic process automation. But he must learn the use of that robotic process. And that is skilling. Therefore, along with theory, skilling is equally important. And this is important in multiple areas. And therefore, multi-disclarity, trans-disclarity, and interdisciplinary. Wonderful. So, what we can now do then is let us take a case study based approach in understanding this problem of me and you want to now design a BTEC in CSE syllabus or BTEC in AI syllabus. That is NIP compliant under the NCRF framework. How do we start? See, if you are designing a course in CSE and you want to start with this concept. Along with your theory, you already had labs. But few places, they were only teaching theory. Teaching theory is not sufficient. If you want the student to really understand and apply that concept practically, along with theory, you have to have a lab. And this has always been there in all over the country. Even when I did my computer science, computer technology from Atty Delhi, we had a corresponding lab. It was not all theoretical. So, those courses are skill-based courses. So, you divide every subject into theory and its application. How do you apply the theory? And those are skill-based courses. Similarly, you can also include internships, projects, apprenticeships around the job training as part of the practical work, part of the skill-based learning, which is occurring, which you are going through. So, then this looks like, let us say, my student stays for one year with me in the first year. And the next year for one full semester, the student can be missing in the campus, but goes out and doesn't appendicep internship. And absolute, we should encourage that. Absolutely. You can do that provided in the first year, you do not teach him the same mundane subjects, which you have been teaching in 12 classes. So, you do not teach him some extension of BSE physics or BSE chemistry. And you give him the actual knowledge of computer science right in the first year. So, that he is prepared enough to go out. He prepared enough to go out. And now, the NCRFL hours, that up to 50 percent of the time, he can spend either in online learning or in apprenticeship or in digital nature of prior learning or even internship. That is unbelievable sir, that 50 percent of the time, which means two full years in a BTEC program, a person can stay outside the campus and do internship, apprenticeship and then come back. But there is a catch and that catch is very important. The catch is that whatever time he spends outside the campus, for each hour of his learning outside the campus, his learning outcomes, and they have to be defined properly. The learning outcomes and their alignment with the overall curricular structure. And then, once he comes back, we have to test the learning outcomes. So, assessment method will have to be specified beforehand, that this is the way I am going to test you. So, he goes out and then he comes back after learning and you again test him. If he passes the assessment, he gets the credits. So, here is where I have thought with some inhibitions about the entire setup, where you are simply assuming a lot of integrity from the teacher side, university side and a lot of integrity from the place where the person goes and works and comes back. We all know a lot of scandals, I am not talking about our country, I am talking about Mother Earth, globally speaking. There are a lot of places where you can go get an internship certificate by paying some money. Although we can keep that a check, it is not easy for us to keep some zones green, some zones red, operational cost for that will be very high on where you can do internship, where you cannot do internship. This will open up Pandora boxes eventually. What is a quick and low budget hack for this? So, that I know the student is learning, the student knows that here is a genuine place and the person there also knows that we should not cheat, it should be value for the times that the student is learning in the campus. Is there the guarantee that there is no check? Now, where in the world? However, when we decide on the companies where the student is sent to learn, the guidelines under NCRF as well as the UGC, they provide for some kind of a joint assessment which is formative. That means, as the student goes along on the training or on the internship, he is given a project and he is putting out whatever he is learning of the project either online or with some times, times, it is being documented. Now, what NCVT has done? NCVT has videographed every assessment. There is a provision that without videography, you cannot issue a certificate. We just imagine a person who is a technician and he claims that all right, he is a good technician and he can repair any kind of car. So, if he repairs, he opens a BMW engine and who totally overalls it, assembles it together, runs it, tests it. And all this is done in front of camera and that camera is not handled by him, it is handled by an assessor which is duly appointed by the awarding body which awards the certificate. And then that video is kept forever. So, you imagine, won't you feel that that guy deserves to be given that certificate? Absolutely. Absolutely. So, credibility and integrity of the system will have to be ensured by us. Now, we just cannot presume that, you know, everyone is bad, yes? If everyone is bad, then what is the guarantee that the education system is giving us anything worthwhile? I think that is a nail on the head, where I think we all should pause for a moment and then think, is our education doing what it is expected to do? Again, I repeat, I am talking globally here. I have seen lots of videos from top Ivy League students complaining saying that, I paid so much for a bachelor's degree, I get close to nothing, I can go to a library and read the book, I can watch the YouTube videos, why should I come to the university? The university is not giving me what I thought it would. Now, that being the case, experimenting something like this, I don't think will be damaging. I think all the institute should come out of their close-minded nesiphony and then try out the NCRF framework where up to 50 percent, again, I repeat, whenever I heard this, I missed the world up to here. I think we have to pause there and then understand that it's up to 50, it can even be 10, it can even be 5. Up to 50 percent of the time, the student can't spend outside their bachelor's curriculum looking at, let's say, the place where I had a bachelor's curriculum, that is part of the bachelor's curriculum outside the compounds of his institute. Outside the academics. Perfect. Outside the academics. Correct. And this is the question of engagement. If I engage with my students, that is why students are crying that we are going to an institute and we are engaged. No, there's no engagement by the teacher, there is no engagement by the institute. There is no engagement by peers, there is no engagement by industry. And he says it rightly and the answer lies in this new framework and no harm in trying this as you rightly say. No, no. Then students are loving it. Students are loving it, students are getting jobs. Institutes, some of the institutes may feel shy for initial period, but whoever has adopted this is much happier. In fact, AICT adopted this last year. I see. I see. So, although that's where my next question comes. AICT says it can be adopted, but colleges that are approved by AICT, let's say a state, you technical university, may not know these rules. Of course, it's a matter of time they will get to know. And verse one is the universities that are institutes that are affiliated to these state universities. They absolutely have no clue what is NCRF. So, this tune I think we should of course, apprise them, go and talk to them and then educate them. But over and beyond this, there are some subtle problems that I realized we have, which is not appearingly at the surface level, which is the student teacher ratio. Now, if I realize that, assume in my college, up to 50 percent, I can make my student study as part of internship, apprenticeship, going outside and then learning, or creating some courses on swam swam plus like platforms. I tell my student, please do it up to 50 percent. I will cut short my faculty by 50 percent, or I will engage my faculty for the betterment of my students at a level which need not necessarily be teaching in the classrooms. Now, in this connection, what bothers is the teacher is to student ratio. That's another, how else? That's a very good question and very good observation that we answer that question. There are about 1200 universities in India and there are 55,000 colleges which are affiliated to these 1200 universities and leaving all the INIs that is institute of national importance. They have about 4.34 crore students with about 15 lakh teachers. So, the teacher student ratio works out to be 1 is to 2647. That has been normal ratio which is working out. Now, do you think this ratio is sufficient to teach a technical subject? Now, once we are sending our students out, again these teachers are not sitting back and enjoying in the institute. They are engaging with the student. Along with the industry, they are constantly monitoring the student and they are also engaged in the firmative assessment of this student. It's not easy to create a project which is outcome based, creating that project itself is going to take a lot of time. Similarly, there are number of courses which the institute can run inside the academy. Inside the academy, we can run number of courses. And these are skill based courses. These courses can be run by the present difficulty only. And wherever there is no expertise available, by the time they acquire those expertise, we can bring people from outside. We can bring people from the industry to teach those courses. So, these teachers are really good to enjoy because in today's times, they are not even enjoying their teaching to that extent unless they are really engaging with the students. So, I do not see any challenge there. And there are no there are going to be no attrition I suppose. Because that number could never go down. As it is, it is really against the students. Right, right, already. It's already. So, but then the college should not disincentivize recruitments. Just in case we understand that we can reduce the workload and the faculty. Not at all, workload will not really reduce in that sense because workloads, shape would change. May not be in the classroom, but outside the classroom. Yes, evaluation. And teacher would be much happier. Teacher would be able to learn more, will be able to collect with the rest of the class even more because he is visiting the industry constantly. He is not even coming to the class and just teaching on the blackboard. That's a good point. So, such engagement should be incentivized. Yes. And documented and validated by the institutes. Absolutely. And universities. That again is going to be non-trivial, but maybe we should come out with SOP for that as well. Absolutely. Yes, absolutely. In fact, this has already been taken into account when we are going for NAC accreditation. These are the points which are actually being washed actually. And if some institute is already going for the implementation of NCRF, skill-based courses have been introduced. No, there are some more points. Much more point for that as well as becomes NAC and NBA friendly. I think we have solved the problem. This is that friendly already. This is already included in the NAC. So, one thing I like about NCRF is it's a nice, I would say, fictitious story, but it's a nice one where I imagine the first time I heard it, it was like a H Hiker trying to go around universities taking up courses and using even the model of RPL where you can get your learning validated if not having undoing on the course completely. It feels like a hippie's way of doing one entire bachelor's or master's program. While it sounds very good on the ears, is it practical? Now, can we then have, let's say, digital university where you accrue points on your what is called the academic bank of credits online and show that to get a certificate. Let's say, I run a university. I think nothing should stop me from looking at a student who has done a bunch of courses the hitchhiking way under the NCRF framework, completed everything. So, I am courses properly proctored by by professors. Why can't we give a BTEC degree for such a student? What is what is the gap in what I am thinking? Excellent, excellent question. I will answer this question in two parts. Firstly, some people ask that they want to acquire creditors from various institutions, put them together and then get a degree. So, my question is, why would somebody like to do that? Difference, substance, it could be on account of the transfer of your parents or change of state by you or marriage or some other family issues, because of which. So, there has to be some enabling clause that you can transfer your credit when you have to go and join some other institution. So, that has been provided here. In fact, some of the institutions are fearing that if we allow this, if some institution is not good, then we must transfer from that particular institution, migration from that institution to the other better institution. They will fear in that. But, nothing of that is not happened. Why? Because this is an enabling provision. It does not mean that if somebody has obtained certain number of credits from a university, say in UP or in say West Bengal or Sipanjab, she can come and you can ask IIT Rupert to give him admission, because he wants to get a transfer here. So, the admission rules of that institution would always apply and the quality standards. Admission rules, admission rules contain quality standards also, whether it is through examination or through marks of certain examination or a fresh sort of assessment. So, all that is to be decided by the institute in such. So, let us say we have a hypothetical question then. Let us say me and you start a private university tomorrow. And there we will identify some swam courses. Can I answer that digital university part of that? Perfect. I was trying to answer that was the second part which I wanted to answer. Now, that digital university concept as you explained is absolutely correct. And in fact, the digital university which is being conceived by the government is on those lines only. I think I feel it should be possible for a student to get few credits from one IIT and few credits from IIT and hands on in the third institution. And then convert that whole thing into a degree, package that whole thing into a degree. Now, whether it could be a BTEC degree or it could be a BSE degree or it could be BAA degree, that would be decided by the curricular structure. Because curricular structure would say whether hands on work is required for the degree or not. If no hands on work is required or if that hands on work can be done online, yes the degree can be given online. Like for example, IIT Madras is giving BTEC degree in data sciences. So, in that case they are holding the examination offline while the entire teaching is online. They are providing materials, the teaching is done and the students are appearing offline. So, that does not mean that their quality is diluting. Those examinations are really tough. They are even tougher than the examinations for the normal student, regular student. So, yes this is a way, this should be done. And if India wants to achieve a GR gross animal enrollment ratio of 50 percent by 2035, whereas in last 75 years, till date we have achieved only 27 percent, 27 percent to 50 percent in just 11 years, this is the way out. So, we have to do it. So, therefore, NCRF is just a nibbling framework for achieving that much. Correct. So, they are my question about a slightly digress and then talk about the attention span of a student. Each hiking actually requires sound school, but requires a lot of willpower from the part of the student. I agree. So, with the declining attention span of her Gen Z. How is this even closer to reality? Can even with a university set up with the attendance being made mandatory, we see attention span being very low for students. And now we are saying you can go around and then take credits. Maybe 5 percent of the Janta in our country are capable of doing this. Do you think it is a Herculean task that we are proposing? You have got the spirit of the question, right. So, is this even feasible? Number one, yes it is feasible. Number two, we must also work on the attention span of her students. And that must start from the school, school syllabus. The way we teach them in school, the way we engage them in schools. Look at Australia, they are just bringing a law, they are banning social media or anybody who is below 16. Very nice, right. And the law has just come. It was just being discussed in the media this week only. So, progressively countries are realizing that we have to ban this culture of short videos, short spans. In few houses I have learnt and they have banned shorts for young kids. Because if they see shorts, their attention span is going to be 30 seconds only. This is where we learn from our Indian knowledge system, how to improve our concentration, how to sit for long hours, how to start reading a book and finish a book, how to make a pressy out of the book, how to share your learnings from the book with others. This will have to be taught to every student by all of us together in schools, in colleges, in universities, everywhere. And this is a universal problem. Not only that India is grappling with this problem, St. Howard, entire world, right. And people are trying to find their own solution. Sir, here is where I think there is a foot foot thought, where I will try to slightly disagree, almost 90 percent I agree with what you said. But slightly disagree there that I actually see a good news in that attention span related discussion that we had just now, where an angster cannot watch a 10 minute video. But he can watch YouTube shots for three full hours. I see a bad news in the first part, good news in the second. Now, we in our lab are trying to design if we can give lessons through YouTube shots, but in continuous capsules. For instance, even in our own guru culture, we would start off with first line of hours. We say the guru will say, yeah, A1 way the student will say, yeah, A1 way the twice. And then he will go to the next line. Again, they should say twice. He would give it in pieces of one one line each. I see that as as the wonderful thought. But I think the students wants to see the short videos, not in these. Successful. Successful. An amendment. Now, he wants every video different. And if he is not getting that video different, even a two and a half year old child will just switch it off and switch it on again. So that the algorithm will throw different kinds of videos every time. That is where the problem lies. That is where the problem lies. The algorithm is wicked there. We fix that with a nice algorithm and use fast food to deliver nutritious food. Absolutely. We can. In fact, that is the smart way of doing it. I'm glad that you agree because it's a big project we are trying to see if we can deliver. You will you will be able to deliver. In fact, we are also working in Karamjogi Bharat on the attention span of the employees. And this is our conclusion that any employee will not be able to spend more than seven and a half minutes of attention span on one particular concept. And even in that seven and a half minutes that concept is broken into three parts of two minutes each followed by 30 seconds of some kind of question or some kind of engagement on that two minutes part. And that is how it is about seven and a half eight minutes total. And then you can give him in one setting you can at most give him three concepts. So in all about 20 21 minutes maximum 22 minutes. And that should be sufficient. Sometimes he takes these 22 minutes in three settings. And he repeats it three times. Wonderful. I think such pedagogy should be studied further and right. So in that connection there are people who can do it fast, who are slow bloomers. In the era where we are slowly talking about slow productivity and the importance of it, we also discuss about educational acceleration. This looks a little, I mean there is a huge disparity between talking about educational acceleration on one side and talking about slow productivity on the other. I am confused here when we when we when we end up there are some gifted minds always. And we are talking about gifted minds at different strokes for different forms absolutely absolutely. So that is not for all not for all we have to cater to such students as well. So for example in Olympia arts means that this student is ahead of others in one particular subject. Olympia arts in math, Olympia arts in physics, right. Okay. And then we also talk about hegathons. Student is so smart that he is able to solve practical issues, practical problems using his present knowledge, his knowledge of ideas and technology. And that too in a limited time that is the hekathan is all about. So in a limited time you are given a problem and you are able to come up with the most feasible solution which is closest to the real solution. So you crack the akathan. So they should also be criticized and that is what NCRF allows. So I am coming there that is the most enticing part of NCRF, akathan being created. But let me just move around the educational acceleration topic for a minute. You are saying that a student if he is really smart should be encouraged to finish his BTEC degree in two years without loss of I mean high quality organic knowledge. Absolutely. Absolutely. If he is able to achieve those learning outcomes, whatever I have been defined for the award of the degree and you properly test those learning outcomes through a rigorous method. Why not? Why not? That is a very good thought. Why is it that we have not thought about all these things worldwide. So far why? See why this was not even thought about because the framework never existed and in education your things are last to change. Because in education most people they go by the proven path especially in teaching not in the search. Right? And now NCRF, since NCRF allows all these possibilities, all these options are now open. So now these options are being tried by various institutions. And I have met number of vice-versa-sales. In fact as part of our exercise to take this NCRF to various universities, I along with the chairman Julesy, we have addressed various conferences in various corners of the country. All five places, all four regions and the central regions, we invited about 200 vice-versa-sales at each place. And we conducted one-day workshop, addressed them and addressed all their questions. We did that and that has been done not only once, but at least twice in the meantime, I say. Also we have engaged with all the directors of the IITs, all the directors of IIMs, all the directors of NITs, all ISR directors and IANIs. We have engaged with the best of the engineering colleges. That has already been done. And now we are also making it part of the teacher's training program. In NCRF, we are trying to create courses on this, around this and teach us a part of the teachers. In NCRF, awareness basically. Yes, basically. It is a small document, it is a 95-page document. And our teachers are so intelligent, they are so evolved. They can pick quickly. They can pick quickly. If they want to implement, every solution is there in NCRF. And if somebody doesn't want to implement, then he is so genius that he will always find a way to create impedance. Yes. Although there is a beautiful SOP written there, as I see, I still feel this NCRF framework is more of a no-rule rule. So beautifully, people are removing all possible handcuffs that education has converged to. But with that comes a problem of a typical academician, let's say like me, I've been into academics in the past 20 years, I think this is the way it should be done. And I close my ears, I don't listen to you. I will look, that's how I say, this is not going to work. How do you convince a checker in the chair? You wonderfully put it because that was the basic objective. To open all the shackles, but those shackles have been opened in a framework. And that framework is very solid. And unless that framework is underneath, this will all be shattered. Now, we don't want it to be shattered. For example, 1200 hours of learning per year has been presumed. 40 credits per year are to be given at school level, at college level, at university level, for skilling institution, for ITI, for polytechnics, everywhere, whether it is a class, eight student or it is a mtex student, it's 40 credits per year. That is the framework. Okay? Learning outcomes will have to be defined. Pre-hat, NCRF level will have to be defined. Pre-hat, number of credits will have to be defined. The poor hand and the assessment method. And then you have to notify this, again this is part of the framework. So, though you have been given full freedom, but freedom within that framework, so that nothing goes haywire. Okay? Now, a commission, if they want to be creative with this framework, there's no limit. So, you're saying hackathons can be credited. It is now that makes it limitless. Absolutely. I would say, I'll call for hackathons of all the colleges in and around. And I'll give them half a credit for participating in the public. Why half a credit? Give them two credits. Give them a problem which can be solved in 120 hours of, with 120 hours of learning. Use as many outcomes as are available from 120 hours learning. Right? If they pass that test, they deserve those credits. So, it's up to you. It's up to the difficulty of the problem. So, this is wonderful, but as and always with very wonderful things, comes with a small caveat. And that is, we teachers, we love going to the class, teaching and coming out, except that one additional thing that we can do is evaluating the unsuscripts. Anything over on top of this is difficult for us, because it's going to be time consuming for us. And we also have our research mandates and things like that. So, don't you think many of these things will at least, appearingly, discourage a young professor, because he will now think, teaching comes with a lot of work to do. I feel the other way around. I feel that this is going to reduce, draw best brains into education. So far, we cannot say that we have been getting best brains in education, except for few top institutions. Reservient people, they have chosen it, not as a preferred option, but as one of the option. Why? Because this was not very creative. It was not very enjoyable. And with NCRF, you make it so open and so enjoyable. Adventureous. However, there would be certain people who would not like to work, for them know it is the same problem either here or in the industry. Anywhere. Anywhere. So, we have to really make them understand that unless they adapt to this, students will not attend their classes. As it is, you will agree with me that most of the students are not sitting in the classes, very difficult to capture them in the classrooms. So, minimize classrooms and increase the... You can capture their attention in the classroom. Unless you engage them in a different way. And NCRF provides you with all the alternative. And you still call it education. You still creditize everything. Correct. Most of it is outside classrooms here. Yes. Whatever we are calling skill-based courses are mostly outside classrooms. Then a connection, what fascinated me is the RPL, which is recognizing prior learning. I assume I am walking into a bachelor's program knowing good amount of physics. It is BSN physics, BSN... I don't know, good amount of physics. I don't want to undergo the mundane, climb the ladder one, CD at a time away. Can I go to the teacher and then tell him, recognize my prior learning, give me a certificate. Absolutely. The recognition of prior learning is the same concept. That if you know the learning outcomes beforehand, why should you be forced to go through the same course and waste your time? Why can't the time be used for making you learn something better where you have interest? So, the recognition of prior learning has been derived in such a way, so that you pre-defined the learning outcomes and you test those learning outcomes with a lot of integrity. Integrity is very important. That is an important parameter, right? That is the most important parameter here. Now, for the first time, UGC is actually thinking about RPL in these situations. So, they have approved the draft guidelines on RPL and those have been put in the public space for public consultation. They are on the table. This provides, I was part of that discussion which happened. And this provides that the first semester, the student cannot have that basic advantage of RPL. He cannot say that I will miss first semester the entire semester as a whole, right? Once he joins the institution, thereafter he can always say that I have done this course or by own or I have done it through self-study, you tell me the learning outcomes and I will sit for your assessment. So, basically there is a tapping also, is it? How many courses you can make it go under? Yes, there is a tapping because otherwise everybody will say that I will do RPL, you have to start from my home page, right? No, it is not that. So, the tapping is that for any course, up to 50% of the time or up to 50% of the credits can be earned either through online courses or through RPL or through apprenticeship and internship. Okay? So, 50% of the time the student is coming to the institute in any case. That is to make sure that we do not value the quality. Maybe in future the academy will change it in favor of the RPL or in favor of the apprenticeship. But as of today, this is what I have been provided. It is a significant number as we can see. And also what I see is let us say, 80-year-old me things that I can afford any kind of education if I can earn it in my 50% part. I am assuming I am doing apprenticeship. It gives me some non-trivial money, at least enough for me to eat and stay in a place. Absolutely, you said it. In fact, there are a certain company or certain institutions who are now working around that they will teach the entire engineering to the student in first one half year. They will not waste their time in basic physics and basic chemistry which we are talking about. And after that they will send them out on apprenticeship for one half year. During that period, the student will earn almost the full salary or at least close to the full salary. And those students are to be placed in highly reputed companies like Google, Microsoft, Oracle, some of the top companies in the world. And those companies will also pay to the institute some money for those placement on those apprenticeships. And that will subsidize the cost of the education for the other students who are here in the institute for two and a half years. Very nice model. Anyone can afford model by that. So, a couple of questions before we end the discussion is work place sincerity, industry related soft skills is something which we hear the industry complain to us that students miss. Again, this is a problem of the globe not just our institute or our country. This is never integrated into the curriculum properly which I think is very important, both sincerity, model dimensions being in place. What are the ways in which NEP with the NCRF framework encourages one to develop soft skills. Number one, the curricular structure now specifically provides for employability skills soft skills and light skills. So, focus really comes on to that earlier this was not there. Secondly, for all skill based courses and specially the courses which are approved by the NCVT under national skill qualification framework, it is provided that at least 10 percent of the time of the student must go on to soft skills. That means, if 600 hours course, so 60 hours will have to be devoted, shall be devoted on these soft skills. It could be more. For a 1 year course, it is 120 hours. So, we picked up the critical learning outcomes, the generic learning outcomes which every student must reign as a graduate. So, the graduate outcomes, generic graduate outcomes, they contain all these soft skills and these soft skills also form part of the level descriptors. Based on a particular soft skill, you will get a particular level in the NCRF. That course will be rated either at level 5.5 or at level 6. So, we make sure that student is given that level of soft skill. Like for example, leadership skill or negotiation skill, conflict resolution skill, cross-cultural competency. There are 6 kinds of thinking itself, there is adaptive thinking, computational thinking, critical thinking, then you have creative thinking, you have innovative thinking, you have design thinking. So, all these 6 kinds of thinking will have to be taught to the two. And they are different, they are different and they are different we use for various purposes. And we have to integrate those skills in the courses. These skills cannot be just taught on a standalone basis. Yes, you can explain it to them. But ultimately, you have to apply this skill to make them perfect. So, that is how it is being done. I see, wonderful. So, it is so you think that the cognitive budget on the teachers will not increase. In fact, it will decrease given that it will give them more time to talk and interact with students. Why have you really bothered about that cognitive time? The teacher is one who is dedicated his life to this tool, Acharya, who has taught his teacher. But for sure, teacher will enjoy it more and his cognitive load will not really increase in that parlance. It is not a road kind of either teaching or learning. It will be a very thoughtful process, cognitive process through which he will also be enjoying along with this tool. He will feel more satisfied and even"""
segmented_output = segment_transcript(transcript)

for idx, segment in enumerate(segmented_output, 1):
    print(f"Segment {idx}:")
    print("\n".join(segment))
    print("-" * 50)
